In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import re

In [3]:
# inputs
samples=snakemake.params.samples
tool=snakemake.params.title

In [4]:
# inputs
sample_name_regex = re.compile("(.*)\.\d+x\..*")
pandora_gene_presence_df = pd.read_csv(snakemake.input.pandora_multisample_matrix, sep="\t")
pandora_gene_presence_df = pandora_gene_presence_df.rename(columns={"Unnamed: 0": "gene_name"})
updated_sample_columns = [sample_name_regex.match(col).group(1) if col != "gene_name" else col for col in pandora_gene_presence_df.columns]
pandora_gene_presence_df.columns = updated_sample_columns
pandora_gene_presence_df

In [5]:
# inputs
bowtie2_gene_presence_df = pd.read_csv(snakemake.input.gene_presence_matrix_based_on_bowtie2, sep="\t")
bowtie2_gene_presence_df

In [6]:
# inputs
gene_length_df = pd.read_csv(snakemake.input.gene_length_matrix, sep="\t")
gene_length_df

In [7]:
merged_dfs = pandora_gene_presence_df.merge(bowtie2_gene_presence_df,
                                            on="gene_name", how="outer", suffixes=("_pandora", "_bowtie"))
merged_dfs

In [8]:
merged_dfs = merged_dfs.fillna(0)
for column in merged_dfs.columns:
    if column != "gene_name":
        merged_dfs[column] = merged_dfs[column].astype(np.int)
merged_dfs

In [9]:
def get_classification(row, sample):
    bowtie_classification = float(row[f"{sample}_bowtie"])
    pandora_classification = float(row[f"{sample}_pandora"])
    
    TP = bowtie_classification == 1 and pandora_classification == 1
    FP = bowtie_classification == 0 and pandora_classification == 1
    FN = bowtie_classification == 1 and pandora_classification == 0
    TN = bowtie_classification == 0 and pandora_classification == 0
    
    if TP: return "TP"
    if FP: return "FP"
    if FN: return "FN"
    if TN: return "TN"
    
for sample in samples:
    merged_dfs[f"{sample}_classification"] = merged_dfs.apply(get_classification, axis=1, sample=sample)
merged_dfs

In [10]:
classification_cols = [col for col in merged_dfs.columns if "classification" in col or col=="gene_name"]
classification_df = merged_dfs[classification_cols]
classification_df

In [11]:
def get_nb_FPs(row):    
    return len([elem for elem in row if elem=="FP"])

gene_and_nb_of_FPs = pd.DataFrame()
gene_and_nb_of_FPs["gene_name"] = classification_df["gene_name"]
gene_and_nb_of_FPs["nb_samples_where_this_gene_is_FP"] = classification_df.apply(get_nb_FPs, axis="columns")
gene_and_nb_of_FPs

In [12]:
gene_and_nb_of_FPs_counted = gene_and_nb_of_FPs.groupby(by="nb_samples_where_this_gene_is_FP").count().rename(columns={"gene_name": "gene_count"})
gene_and_nb_of_FPs_counted.to_csv(snakemake.output.gene_and_nb_of_FPs_counted_data)
gene_and_nb_of_FPs_counted

In [13]:
def get_series_summarizing_gene_classification(normalize):
    all_samples = pd.Series()
    for col in classification_df.columns:
        if col.endswith("_classification"):
            all_samples = all_samples.append(classification_df[col])
    all_samples.name="all_samples"
    return pd.DataFrame([all_samples.value_counts(normalize=normalize)])

gene_classification_summary = get_series_summarizing_gene_classification(normalize=False)
gene_classification_summary_normalized = get_series_summarizing_gene_classification(normalize=True)
gene_classification_summary_normalized.xs("all_samples")
gene_classification_summary_normalized.to_csv(snakemake.output.gene_classification_plot_data)
gene_classification_summary_normalized

In [16]:
def get_plot_gene_classification_binned_by_sample(data, title, xaxis_title, yaxis_title):
    fig = go.Figure(data=[
            go.Bar(name=classification, x=data.index, y=data[classification])
            for classification in data.columns])

    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title, barmode='stack')
    return fig


In [18]:
plot_gene_classification_all_samples = \
    get_plot_gene_classification_binned_by_sample(gene_classification_summary, \
    title=f"Gene classification all_samples - {tool}", \
    xaxis_title="All samples", \
    yaxis_title="Count")
plot_gene_classification_all_samples.write_image(snakemake.output.gene_classification_plot)
plot_gene_classification_all_samples

In [19]:
def get_gene_classification_by_sample(normalize):
    classification_df_summary = pd.DataFrame()
    count_series = []
    for col in classification_df.columns:
        if col.endswith("_classification"):
            count_series.append(classification_df[col].value_counts(normalize=normalize))
    
    return pd.DataFrame(count_series)

gene_classification_by_sample = get_gene_classification_by_sample(normalize=False)
gene_classification_by_sample_normalized = get_gene_classification_by_sample(normalize=True)

gene_classification_by_sample.to_csv(snakemake.output.gene_classification_by_sample_plot_data)
gene_classification_by_sample

In [20]:
plot_gene_classification_binned_by_sample = \
    get_plot_gene_classification_binned_by_sample(gene_classification_by_sample, \
    title=f"Gene classification binned by sample - {tool}", \
    xaxis_title="Sample", \
    yaxis_title="Count")
plot_gene_classification_binned_by_sample.write_image(snakemake.output.gene_classification_by_sample_plot)
plot_gene_classification_binned_by_sample

In [21]:
classification_df_with_gene_length = classification_df.merge(gene_length_df, on="gene_name")
classification_df_with_gene_length

In [22]:
def get_gene_length_category(value):
    if value >= 4000:
        return 4100
    else:
        return (int(value/100)+1)*100

classification_df_with_gene_length["gene_length_category"] = \
classification_df_with_gene_length["gene_length"].apply(get_gene_length_category)
classification_df_with_gene_length.to_csv(snakemake.output.gene_classification_by_gene_length_plot_data, index=False)
classification_df_with_gene_length

In [23]:
gene_names = []
classifications = []
gene_length_categories = []
for _, row in classification_df_with_gene_length.iterrows():
    for col in classification_df_with_gene_length.columns:
        if col.endswith("_classification"):
            gene_names.append(row["gene_name"])
            classifications.append(row[col])
            gene_length_categories.append(row["gene_length_category"])
            
classification_all = pd.DataFrame(data={"gene_name": gene_names, "classification": classifications, "gene_length_category": gene_length_categories})
classification_all

In [31]:
classification_all_grouped_and_counted = classification_all.groupby(by=["classification", "gene_length_category"]).count()
classification_all_grouped_and_counted = classification_all_grouped_and_counted.unstack().fillna(0)
classification_all_grouped_and_counted.to_csv(snakemake.output.gene_classification_by_gene_length_plot_data)
classification_all_grouped_and_counted

In [32]:
import plotly.graph_objects as go

def get_plot_gene_classification_binned_by_length(data, title, xaxis_title, yaxis_title):
    gene_length_categories = data.columns.get_level_values(1)

    fig = go.Figure(data=[
            go.Bar(name=classification, x=gene_length_categories, y=data.xs(classification)) \
                   for classification in data.index],
            )

    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title, barmode='stack')
    return fig

plot_gene_classification_binned_by_length = \
    get_plot_gene_classification_binned_by_length(classification_all_grouped_and_counted, \
    title=f"Gene classification binned by gene length - {tool}", \
    xaxis_title="Gene length", \
    yaxis_title="Count")


plot_gene_classification_binned_by_length.write_image(snakemake.output.gene_classification_by_gene_length_plot)
plot_gene_classification_binned_by_length

In [33]:
def normalize_column(column):
    total = sum(column)
    column = column.apply(lambda value: value/total)
    return column
    
classification_all_grouped_and_counted_normalized = classification_all_grouped_and_counted.apply(normalize_column)
classification_all_grouped_and_counted_normalized.to_csv(snakemake.output.gene_classification_by_gene_length_normalised_plot_data)
classification_all_grouped_and_counted_normalized

In [34]:
plot_gene_classification_binned_by_length_normalized = \
    get_plot_gene_classification_binned_by_length(classification_all_grouped_and_counted_normalized, \
    title=f"Gene classification binned by gene length - Normalised - {tool}", \
    xaxis_title="Gene length", \
    yaxis_title="Proportion")

plot_gene_classification_binned_by_length_normalized.write_image(snakemake.output.gene_classification_by_gene_length_normalised_plot)
plot_gene_classification_binned_by_length_normalized